#### Carga de librerías

In [1]:
import pandas as pd
import numpy as np
import re
import os

# NLP
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Prepocesamiento
from sklearn.preprocessing import MinMaxScaler

# Train-Test, Validación
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

# Regresores
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

# Metricas para regresiones
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score

# Visualizaciones
import plotly.express as px

#### Descarga de librerías adicionales

In [2]:
try:
    import spacy
except:
    %pip install spacy
    !python -m spacy download es_core_news_sm

#### Carga del dataset completo de comentarios

In [ ]:
df_completo = pd.read_csv("Data/100178_Comentarios.csv", sep=";")

#### Extracción de la feature "texto" y el target "karma"

In [ ]:
df = df_completo[["texto", "karma"]]
df.shape

#### Eliminación de valores nulos y conversión de tipo

In [ ]:
print(f"Total de nulos por columna\n{df.isna().sum()}")
df = df.dropna()
df["karma"] = df["karma"].astype("int32")
df.reset_index(inplace=True, drop=True)
df.shape

#### Definición de funciónes auxiliares

In [ ]:
# Cargar de las stopwords en español de nltk
stopwords = nltk.corpus.stopwords.words("spanish")

# Cargar del modelo de lenguaje para español de Spacy
nlp = spacy.load("es_core_news_sm")

# Función para la limpieza de comentarios
def limpieza_texto(texto: str):
    '''
    Función de limpieza de texto que elimina números, signos de puntuación y caractereres no alfanuméricos.\n
    Elimina stopwords, lematiza la entrada y devuelve una cadena de texto con todas las palabras de más de 2 letras. \n
    Necesita definición previa de la lista de "stopwords" y la inicialización del modelo de lenguaje de Spacy en la variable "nlp"
    '''
    texto = re.sub(r'[^a-zA-Z0-9\sáéíóúñçÁÉÍÓÚäëïöüàèìòù]', ' ', texto)
    texto = re.sub(r'\d+', '', texto)

    # Lematización
    comentario_nlp = nlp(texto)
    lemmas = [token.lemma_ for token in comentario_nlp]
    comentario_stopwords = ' '.join(lemmas)
    
    # Elimnación de Stopwords y palabras de menos de 3 letras
    tokens_limpios = [palabra for palabra in comentario_stopwords.split(" ") if palabra.lower() not in stopwords and len(palabra) > 2]
    
    return ' '.join(tokens_limpios)

# Función para eliminar hapaxes
def eliminar_palabras(texto: str, lista: list[str]):
    '''
    Elimina de un texto dado las palabras contenidas en la lista de palabras provista
    '''    
    texto_limpio = ' '.join([palabra for palabra in texto.split(" ") if palabra not in lista])
    return texto_limpio

def DistribucionPalabras_RiquezaLexica(datos: str):
    '''
    Función para mostrar la riqueza léxica de un texto.\n
    Retorna la distribución de palabras (nltk.FreqDist).
    
    '''
    # Tokenización del texto y generación de la distribución
    tokens = nltk.word_tokenize(text = datos, language = "spanish") 
    texto_nltk = nltk.Text(tokens = tokens)
    distribucion = nltk.FreqDist(samples = texto_nltk)

    # Obtención de la riqueza Léxica
    total_palabras = len(tokens)
    palabras_diferentes = len(set(tokens))
    riqueza_lexica = round((palabras_diferentes / total_palabras) * 100, 2)
    
    # Obtención de hapaxes
    hapaxes = distribucion.hapaxes()

    print(f"Total de palabras: {total_palabras}")
    print(f"Palabras diferentes: {palabras_diferentes}")
    print(f"Riqueza Lexica: {riqueza_lexica}% palabras distintas")
    print (f"Número total de hapaxes: {len(hapaxes)}")

    return distribucion

### Limpieza del texto
Se eliminan también los posibles nulos resultantes tras la limpieza

In [ ]:
df["texto"] = df["texto"].apply(limpieza_texto)
df = df.dropna()
df.reset_index(inplace=True, drop=True)
df.shape

##### Exportacion Intermedia de Datos y obtención del texto completo

In [ ]:
# df.to_csv("Data/100K_Comentarios_TrasLimpieza.csv", sep=";", encoding="utf-8",  index=False)
try:
    with open("Data/Texto_Comentarios_100K.txt", mode="r", encoding="utf-8") as file:
        texto_completo = file.read()
except:
    texto_completo = ""
    try:
        os.mkdir("Data")
    except:
        pass
    with open("Data/Texto_Comentarios_100K.txt", mode="w", encoding="utf-8") as file:
        for fila in df["texto"].values:
            file.write(fila + " ")
            texto_completo += fila + " "

### Generación y graficado de la Distribución de palabras y muestra de la riqueza léxica

In [ ]:
distribucion = DistribucionPalabras_RiquezaLexica(texto_completo)
distribucion.plot(20)

### Obtención de hapaxes y aplicación de la función de eliminación

In [ ]:
hapaxes = distribucion.hapaxes()
df["texto"] = df["texto"].apply(lambda x: eliminar_palabras(x, hapaxes))

#### Exportación intermedia de datos

In [ ]:
try:
    df = pd.read_csv("Data/100KComentarios_Procesados.csv", sep=";", encoding="utf-8")
except:
    try:
        os.mkdir("Data")
    except:
        pass
    df.to_csv("Data/100KComentarios_Procesados.csv", sep=";", encoding="utf-8", index=False)

try:
    with open("Data/Texto_Comentarios_100K_Procesados.txt", mode="r", encoding="utf-8") as file:
        texto_completo_procesado = file.read()
except:
    texto_completo_procesado = ""
    
    try:
        os.mkdir("Data")
    except:
        pass
    
    with open("Data/Texto_Comentarios_100K_Procesados.txt", mode="w", encoding="utf-8") as file:
        for fila in df["texto"].values:
            file.write(fila + " ")
            texto_completo_procesado += fila + " "

### Distribución tras la eliminación de hapaxes

In [ ]:
distribucion = DistribucionPalabras_RiquezaLexica(texto_completo_procesado)
distribucion.plot(20)

In [ ]:
distribucion.hapaxes()

### Distribución de la variable objetivo "karma"
Se observa que la variable toma valores negativos y sufre un desbalanceo de la distribución.

In [ ]:
px.histogram(df["karma"], marginal="box")

### Tratamiento de outliers

1º - Escalado para eliminar valores negativos

2º - Normalización con la función logaritmica. Se añade desplazamiento para evitar el logaritmo de 0.

In [ ]:
karma_scaler = MinMaxScaler()
karma_escalado = karma_scaler.fit_transform(df["karma"].values.reshape(-1,1))
karma_normalizado = np.log(karma_escalado + 1)
px.histogram(karma_normalizado, marginal="box")

Obtención de los cuantiles, el rango intercuartil y los bigotes

In [ ]:
karma_Q1 = pd.DataFrame(karma_normalizado)[0].describe()["25%"]
karma_Q3 = pd.DataFrame(karma_normalizado)[0].describe()["75%"]

karma_RIC = karma_Q3 - karma_Q1

bigote_superior = karma_Q3 + 1.5*karma_RIC
bigote_inferior = karma_Q1 - 1.5*karma_RIC

Transformación inversa (exponenciación) y desescalado

In [ ]:
bigote_superior = int(round(karma_scaler.inverse_transform((np.exp(bigote_superior) - 1).reshape(1,-1))[0][0],0))
bigote_inferior = int(round(karma_scaler.inverse_transform((np.exp(bigote_inferior) - 1).reshape(1,-1))[0][0],0))
print(f"Bigote superior: {bigote_superior}\nBigote inferior: {bigote_inferior}")

#### Eliminación de Outliers de acuerdo a los bigotes

In [ ]:
df = df[(df["karma"] >= bigote_inferior) & (df["karma"] <= bigote_superior)]
df.reset_index(inplace=True, drop=True)

#### Conteo y Vectorizado de las palabras

In [ ]:
# Inicializamos un objeto CountVecrtorizer()
count_vectorizer = CountVectorizer()

# Entrenamos el modelo y transformamos los datos.
bag = count_vectorizer.fit_transform(df["texto"])

bag

#### Transformación de la matriz de palabras por frecuencias

In [ ]:
# Inicializamos un objeto Tfidf
tfidf = TfidfTransformer()

# Entrenamos el Tfidf y transformamos la variable bag
bag_tfidf = tfidf.fit_transform(bag)
bag_tfidf

#### Creación de los grupos de train/test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bag_tfidf, # X
                                                    df["karma"].values, # y
                                                    test_size = 0.3, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

#### Escalado del target para eliminar valores negativos

In [ ]:
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1,1))
y_test = y_scaler.transform(y_test.reshape(-1,1))

#### Transformación logarítmica para acercarse a una distribución normal (gaussiana)

In [ ]:
y_train = np.log(y_train + 1)
y_test = np.log(y_test + 1)

#### Listado inicial de modelos

Se descartan RadiusNeighborsRegressor, RandomForestRegressor y SVR por su coste computacional

In [ ]:
modelos = [LinearRegression(),
            KNeighborsRegressor(),            
            DecisionTreeRegressor(),            
            AdaBoostRegressor(),
            GradientBoostingRegressor()]

#### Entrenamiento y predicción con todos los modelos y generación de métricas

In [ ]:
datos_modelos = list()

for model in modelos:
    print (f"    #### Modelo: {str(model)} ####")
    model.fit(X_train, y_train.reshape(-1,))
    yhat = model.predict(X_test)

    y_test_inv = np.exp(y_test) - 1
    yhat_inv = np.exp(yhat) - 1

    y_test_inv = y_scaler.inverse_transform(y_test_inv)
    yhat_inv = y_scaler.inverse_transform(yhat_inv.reshape(-1,1))

    #Metricas
    mae = mean_absolute_error(y_test_inv, yhat_inv)
    mse = mean_squared_error(y_test_inv, yhat_inv)
    rmse = root_mean_squared_error(y_test_inv, yhat_inv)
    r2 = r2_score(y_test_inv, yhat_inv)
    
    datos_modelos.append([str(model).strip("()"), model, mae, mse, rmse, r2])
    print(f"-MAE: {mae}  -MSE: {mse}  -RMSE: {rmse}  -R2: {r2}")

df_modelo = pd.DataFrame(data = datos_modelos, columns = ["name", "model", "mae", "mse", "rmse", "r2"])
df_modelo.sort_values("mae", ascending = False)

#### Determinamos el modelo con mejores métricas - GradientBoostingRegressor
Imprimimos las métricas como referencia

In [ ]:
model = GradientBoostingRegressor()
print (f"#### Modelo: {str(model).strip('()')} ####")
model.fit(X_train, y_train.reshape(-1,))
yhat = model.predict(X_test)

y_test_inv = np.exp(y_test) - 1
yhat_inv = np.exp(yhat) - 1

y_test_inv = y_scaler.inverse_transform(y_test_inv)
yhat_inv = y_scaler.inverse_transform(yhat_inv.reshape(-1,1))

#Metricas
mae = mean_absolute_error(y_test_inv, yhat_inv)
mse = mean_squared_error(y_test_inv, yhat_inv)
rmse = root_mean_squared_error(y_test_inv, yhat_inv)
r2 = r2_score(y_test_inv, yhat_inv)

print(f"MAE: {mae}\nMSE: {mse}\nRMSE: {rmse}\nR2: {r2}")

 #### Procedemos a la validación mediante KFold

In [ ]:
# Elegimos realizar 5 folds.
Validacion_kfold = KFold(n_splits = 5)

# Creación de listas para los resultados generales y parciales
lista_targets = list()
lista_predicciones = list()
lista_mae = list()
lista_rmse = list()

i = 0

for train_index, test_index in Validacion_kfold.split(df):
    i += 1
    print (f"#### Fold {i} ####")
    
    # Creación de los conjuntos de Train y Test
    X_train, X_test = bag_tfidf[train_index], bag_tfidf[test_index]
    y_train, y_test = df["karma"][train_index].values, df["karma"][test_index].values

    # Normalización de la columna objetivo en ambos conjuntos
    y_scaler = MinMaxScaler()
    y_train = y_scaler.fit_transform(y_train.reshape(-1,1))
    y_test = y_scaler.transform(y_test.reshape(-1,1))
    y_train = np.log(y_train + 1)
    y_test = np.log(y_test + 1)
    
    # Creación y entrenamiento del Modelo    
    GradientBoosting = GradientBoostingRegressor()
    GradientBoosting.fit(X_train, y_train.reshape(-1,))

    # Prediccion
    yhat = GradientBoosting.predict(X_test)

    y_test_inv = np.exp(y_test) - 1
    yhat_inv = np.exp(yhat) - 1

    y_test_inv = y_scaler.inverse_transform(y_test_inv)
    yhat_inv = y_scaler.inverse_transform(yhat_inv.reshape(-1,1))

    # Metricas
    mae = mean_absolute_error(y_test_inv, yhat_inv)    
    rmse = root_mean_squared_error(y_test_inv, yhat_inv)
    print(f"MAE: {mae}\nRMSE: {rmse}\n")

    # Se añaden las métricas a listas
    lista_targets.extend(y_test_inv)
    lista_predicciones.extend(yhat_inv)
    lista_mae.append(mae)
    lista_rmse.append(rmse)

# Se calcula el desempeño general y se imprimen los rangos de las métricas observadas
print(f"GENERAL MAE:\t {mean_absolute_error(lista_targets, lista_predicciones)}")
print(f"GENERAL RMSE:\t {root_mean_squared_error(lista_targets, lista_predicciones)}\n")

print(f"Min de MAE: {np.array(lista_mae).min()}")
print(f"Media de MAE: {np.array(lista_mae).mean()}")
print(f"Max de MAE: {np.array(lista_mae).max()}\n")

print(f"Min de RMSE: {np.array(lista_rmse).min()}")
print(f"Media de RMSE: {np.array(lista_rmse).mean()}")
print(f"Max de RMSE: {np.array(lista_rmse).max()}\n")


Las métricas resultan muy similares en todos los folds.

Se presume que el modelo es consistente independientemente de lo datos de entrenamiento/test

### Mejora de parámetros mediante GridSearchCV

Se ejecuta el GridSearchCV con los parámetros establecidos

In [ ]:
y_scaler = MinMaxScaler()
y = np.log(y_scaler.fit_transform(df["karma"].values.reshape(-1,1)) + 1 )

# Modelo
GradientBoosting = GradientBoostingRegressor()

# Parametros a iterar
parametros = {
    'n_estimators': [200, 300],
    'learning_rate': [0.01, 0.05, 0.2],
    'max_depth': [4, 5, 6],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]    
}

# Metricas
scorers = ["neg_mean_absolute_error", "neg_mean_squared_error"]

# GridSearchCV
grid_solver = GridSearchCV(estimator  = GradientBoosting,
                           param_grid = parametros,
                           scoring    = scorers,
                           cv         = 2,
                           refit      = "neg_mean_absolute_error",
                           n_jobs     = -1,
                           verbose    = 2)

# Resultados
model_result = grid_solver.fit(bag_tfidf, y)

#### Se imprime el resultado de la métrica a maximizar y los mejores parámetros

Al tratarse de una métrica de error, la cual se trata de minimizar típicamente, el GridSearchCV realiza la multiplicación por -1 a la hora de calcularla y por tanto poder maximizar la misma.

In [ ]:
resultado_real = np.exp(model_result.best_score_) - 1
resultado_real = y_scaler.inverse_transform(resultado_real.reshape(1,-1))
print(f"Media de MAE: {model_result.cv_results_["mean_test_neg_mean_absolute_error"].mean()}")
print(f"Media de MSE: {model_result.cv_results_["mean_test_neg_mean_squared_error"].mean()}")

print(f"Mejor resultado tras el escalado: {resultado_real}")
print(f"Mejores parámetros:\n{model_result.best_params_}")

### Se establecen los parámetros obtenidos y se entrena el modelo

In [ ]:
modelo_final = GradientBoostingRegressor(learning_rate=0.05,
                                         max_depth=4,
                                         min_samples_leaf=4,
                                         min_samples_split=10,
                                         n_estimators=300)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bag_tfidf, # X
                                                    df["karma"].values, # y
                                                    test_size = 0.2, random_state = 42)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1,1))
y_test = y_scaler.transform(y_test.reshape(-1,1))

y_train = np.log(y_train + 1)
y_test = np.log(y_test + 1)

print (f"#### Modelo: {str(modelo_final)} ####")
modelo_final.fit(X_train, y_train.reshape(-1,))
yhat = modelo_final.predict(X_test)

y_test_inv = np.exp(y_test) - 1
yhat_inv = np.exp(yhat) - 1

y_test_inv = y_scaler.inverse_transform(y_test_inv)
yhat_inv = y_scaler.inverse_transform(yhat_inv.reshape(-1,1))

#Metricas
mae = mean_absolute_error(y_test_inv, yhat_inv)
mse = mean_squared_error(y_test_inv, yhat_inv)
rmse = root_mean_squared_error(y_test_inv, yhat_inv)
r2 = r2_score(y_test_inv, yhat_inv)

print(f"MAE: {mae}\nMSE: {mse}\nRMSE: {rmse}\nR2: {r2}")

### Creación de textos de prueba

In [ ]:
textos_prueba = ["Científicos de la Universidad de Australia Occidental y Kelpie Geosciences en el Reino Unido filmaron en vídeo un raro calamar de aguas profundas a una profundidad de más de un kilómetro bajo el agua. El calamar anzuelo de aguas profundas es uno de los calamares de aguas profundas más grandes y es famoso por tener dos fotóforos muy grandes en el extremo de dos de sus brazos, que producen brillantes destellos bioluminiscentes para asustar y desorientar a sus presas cuando caza. Estos son los fotóforos más grandes conocidos en el mundo natural.",
                 "Aguasprofundas, calamar, ningún chiste sobre un desuellamentes o Illithid en los primeros 10 comentarios.... Algo falla aquí. ¿Ha subido la media de edad?",
                 "Alaa es una niña. Vive (o más bien sobrevive) en la franja de Gaza. Junto con su familia (los que aún viven) ha sido desplazada a Rafah. Sí, esa ciudad a donde Israel obligó a refugiarse a los que …",
                 "#3 Pero si para la ultraizquierda todo el que no sea como ellos, es 'fascista'. Aquí en España los desquiciados votantes socialcomunistas llamaban 'trifachito' a PP, Cs y VOX. Los 'demócratas', la izquierda que expresa públicamente que quiere prohibir todo lo que no sea socialcomunista.",
                 "Estaremos todos de acuerdo en una persona que razona así no puede estar en contacto con niños,si?"]

Serie_pruebas = pd.DataFrame(textos_prueba, columns=["texto"])

#### Se aplica la limpieza del texto, se transforma con los objetos previamente entrenados y se realiza la predicción

In [ ]:
Serie_pruebas["texto"] = Serie_pruebas["texto"].apply(limpieza_texto)
bag_prueba = count_vectorizer.transform(Serie_pruebas["texto"])
bag_tfidf_prueba = tfidf.transform(bag_prueba)
resultado_prueba = modelo_final.predict(bag_tfidf_prueba)

#### Se realiza la transformación inversa y el desescalado y se añaden los resultados al los textos de prueba

In [ ]:
resultado_prueba_inv = np.exp(resultado_prueba) - 1
resultado_prueba_inv = y_scaler.inverse_transform(resultado_prueba_inv.reshape(-1,1))
Serie_pruebas["karma"] = resultado_prueba_inv
Serie_pruebas